# # MM pipeline: Run the DeLTA tracking model over all folders

This has originally been provided by Owen O'Connor and modified by Julian Baer to work with the pipeline to analyse mother machine images analysed with JB's pipeline.
The delta post-processing script that creates a nice .csv that I import into R for analysis was provided by Simon van Vliet. The outputs are as followed:

In the data frame we now have the following lineage information:

- `id_seg`: the ordinal lineage id assigned by delta (contains cell+offspring), this index (offset by 1) in label image. Don't use this unless you need to interface wit label image
- `id_cell`: a unique id for each cell, from birth to division. Always use this to access cell lineages.
- `id_par`: the `id_cell` number of a cell's parent
- `id_colony`: each cell in first frame is assigned a unique `id_colony` which is shared with all it's offspring. (e.g. use this to separate different colonies)
- `id_d1`: the `id_cell` number of a cell's first offspring (old-pole)
- `id_d2`: the `id_cell` number of a cell's second offspring (new-pole)
- `id_sib`: the `id_cell` number of a cell's sibling
- `generation`: cell generation: 0 for cells present in first frame, 1 for first generation of offspring, etc.
- `age`: frame number relative to birth of cell, i.e. it goes from 0:T where T is life time of cell

### Setup

Specify paths and variables in the s0_writeconfig.mat file:
- `deltapath`  as I use a locally modified delta version that is not installed via conda or pip, I append the path to where I have that delta package stored and load from there. Please modify accordingly!! This modified version is included in the github repo.
- `configpath` DeLTA uses it's own seperate config file. This is the path to the storage of this config file. This needs to point to the tracking model (hdf5) file. This hdf5 is large and not included in the github repo. You can download my mother-machine 2D model for cocci here: LINK
- `configname` name of the DeLTA config file
- `imageprototype` structure of the image naming scheme, all the ID numbers in it and the number format of the IDs
- `orderprototype` corresponding structure of IDs. *p* = position; *c* = color channel; *t* = frame
- `image_heigth` and `image_width` pad images to this size if they're smaller. 256x256 is also the size the tracking model has been trained for and it cannot be smaller than that. Never tried with bigger images... Your task! :)
- `cropimages` a parameter in the creation of the position object of delta. No idea if this has an effect on tracking or only on their segmentation model
- `driftcor` a parameter in the creation of the posiiton object of delta. Maybe you can use that?
- `saveformats` you can save movies for the tracking, data in pickle or .mat formats (native delta outputs). Additionally, I anyway save a .csv for each movie with some (for me) easier-to-read formating.
- `clearsaves` If True, delete all detected delta output before attempting to track. Useful if you're troubleshooting and want to make sure all previous attempts are erased
- `savemovie` If True, save the Delta movie
- `savemoviefreq` frequency of movie saving.
- `features_list` delta features to track.



### Load main config file. Adapt directory

In [1]:
configdir = 'G://GitHub/microfluidics-image-processing/MM_pipeline';
mainconfigname = 'config_example_matched';

if not mainconfigname.endswith('.json'):
    mainconfigname += '.json'
    
if not configdir.endswith('/'):
    configdir += '/'

import json
# Read JSON data
with open(configdir+mainconfigname, 'r') as file:
    data = json.load(file)

# Assign each key-value pair as a variable
for key, value in data.items():
    globals()[key] = value

### Check if the DeLTA config path is correct

In [2]:
print(configpath)

G://GitHub/microfluidics-image-processing/delta_config/


### Load various packages

In [4]:
import numpy as np 
from pathlib import Path
import cv2
import sys
import pathlib
sys.path.append(deltapath)
import glob

import delta
import delta.utilities as utils
from delta.utilities import cfg
from delta.delta_postprocess import delta_to_df #contains all the code to convert delta object

#import napari
import matplotlib.pyplot as plt
import skimage
from skimage.segmentation import clear_border
from scipy import ndimage 
import os
from tifffile import imread, imwrite
from tqdm import tqdm

import pandas as pd
from datetime import date

def to_str(posixpath):
    return str(posixpath.resolve())

cfg.load_config(to_str(pathlib.Path(configpath + configname)))
cfg.save_format = saveformats

Loading configuration from: G:\GitHub\microfluidics-image-processing\delta_config\config_2D_azimages.json


### Load in meta file and display head. Check if correct

In [5]:
meta = pd.read_csv(os.path.join(masterdir,metacsv))
replicates = meta.replicate.unique()
meta.head()

,date,replicate,chip,pos,channel,Process,replicate2,Process2,rep2startdifferencemin,rep2firstframe,...,StageY,PxinUmX,PxinUmY,register,stardist,stardist_data,stardist_data_cor,stardist_fails,delta,delta_fails
0,22.08.2024,r02,c1,1,2,1108,NaN,NaN,NaN,NaN,...,-36507.72605,0.065,0.065,Done,Done,NaN,NaN,NaN,NaN,NaN
1,22.08.2024,r02,c1,2,3,1113,NaN,NaN,NaN,NaN,...,-37249.40072,0.065,0.065,Done,Done,NaN,NaN,NaN,NaN,NaN
2,29.08.2024,r04,c1,1,2,1784,r04b,1847.0,145.0,41.0,...,-38615.71153,0.065,0.065,Done,Done,NaN,NaN,NaN,NaN,NaN
3,NaT,r06,c1,1,3,2075,r06b,2151.0,65.0,10.0,...,-37305.10401,0.065,0.065,Done,Done,NaN,NaN,NaN,NaN,NaN
4,NaT,r06,c2,2,1,2093,r06b,2169.0,65.0,10.0,...,-37185.52962,0.065,0.065,Done,Done,NaN,NaN,NaN,NaN,NaN


# The actual tracking loop

In [ ]:
# check number of channels and add intensity quantification to features list
if n_channel > 1:
    features_list.append("fluo1")

if n_channel > 2:
    features_list.append("fluo2")

# Initialize counter
saved_movies = 0

# Create a blank reference image based on drift correction setting
new_image_width = image_width
new_image_height = image_height
ref = None if driftcor else np.zeros((image_height, image_width), dtype=np.float32)

# Process each row in the meta file
for i in range(meta.shape[0]):
    fails = []  # To record any failures
    
    
    # Skip conditions
    row = meta.iloc[i]
    if row['delta'] == 'Done' or row['Exclude'] in ['excl', 'skip'] or row['stardist'] != 'Done' or row['dt'] > 3 or isinstance(row['replicate2'], str):
        continue
        
    # Setup directories
    main_folder = os.path.join(masterdir, savedirname, row['replicate'], 'Chambers')
    save_directory = os.path.join(main_folder, deltasavename)
    os.makedirs(save_directory, exist_ok=True)

    # Clear saved files if needed
    if clearsaves:
        for pattern in ['*Pos{}*mp4', '*Pos{}*csv']:
            for file in Path(save_directory).glob(pattern.format(str(row['pos']).zfill(2))):
                os.remove(file)

    # Current position folder
    curr_dir = os.path.join(main_folder, f'Pos{round(row["pos"]):02d}')

    # List all image folders
    chamber_folders = [f.path for f in os.scandir(curr_dir) if f.is_dir() and 'Chamb' in f.name]


    # Process each chamber folder
    for chami, chamber_folder in enumerate(tqdm(chamber_folders, desc=f"{row['replicate']}, Pos {round(row['pos']):02d}")):
        try:
            # Initialize reader and position object
            reader = delta.utils.xpreader(Path(curr_dir), fileorder=orderprototype, prototype=imageprototype)
            position_nb = chami

            # rois set to 0
            rois_nb = 0

            #print(chamber_folders[chami])
            # get list of all images
            unprocessed = sorted(glob.glob((chamber_folders[chami]) + "/*.tif"))

            # create the delta position object
            pos = delta.pipeline.Position(
                position_nb=position_nb,
                reader=reader,
                models=delta.utils.loadmodels(),
                drift_correction=driftcor,
                crop_windows=cropimages
                )

            # features to extract
            features = tuple(features_list)

            # run delta pre-processing                       
            pos.preprocess(rotation_correction=delta.config.rotation_correction, reference = ref)


            # get list of frames/timepoints and go over all images and segmentation files
            # to pad them to 256x256.
            # for the segmentation images, transform them to binary masks and make sure
            # that there's always a gap between cells
            frames = [f for f in range(pos.reader.timepoints)]
            for trans_frameN in unprocessed:
                # read in current image
                trans_frame = cv2.imread(str(trans_frameN),cv2.IMREAD_ANYDEPTH)
                # scale it to [0, 1]
                trans_frame = utils.rangescale(trans_frame, rescale=(0, 1))
                # get old image size
                old_image_height, old_image_width = trans_frame.shape

                # if it is bigger, crop it. Not sure if that works well..?
                if new_image_height<old_image_height:
                    dif = old_image_height - new_image_height
                    crop_img = trans_frame[dif:old_image_height, :]
                else:
                    crop_img = trans_frame
                # reevaluate size
                old_image_height, old_image_width = crop_img.shape
                # init an empty image
                color = 0
                result = np.full((new_image_height,new_image_width), color, dtype=np.float32)
                # compute center offset
                x_center = (new_image_width - old_image_width) // 2
                y_center = (new_image_height - old_image_height) // 2
                # copy img image into center of result image
                result[y_center:y_center+old_image_height, 
                       x_center:x_center+old_image_width] = crop_img
                # append these images to the position object
                pos.rois[rois_nb].img_stack.append(result)

            # same for the segmentation images
            seg_fps = sorted(glob.glob((chamber_folders[chami] + '/seg_sd2') + "/*.tif"))
            for seg_fp in seg_fps:
                # read image
                seg = cv2.imread(str(seg_fp),cv2.IMREAD_ANYDEPTH)
                # get size, crop if needed
                old_image_height, old_image_width = seg.shape
                if new_image_height<old_image_height:
                    dif = old_image_height - new_image_height
                    crop_img = seg[dif:old_image_height, :]
                else:
                    crop_img = seg

                old_image_height, old_image_width = crop_img.shape
                color = 0
                # format new image
                if (np.amax(crop_img)>0):
                    crop_img *= (255.0/crop_img.max())
                crop_img = crop_img.astype('int32')

                # get the boundaries of labels, set these to 0
                mask = skimage.segmentation.find_boundaries(crop_img, connectivity=1, mode='outer', background=0)
                crop_img[mask] = 0

                # remove cells that touch upper and lower parts of the image. These are difficult to track
                # you may remove that part
                bordermask = np.full((old_image_height,old_image_width), 1, dtype=np.float32)
                bordermask[:,0:2] = 0
                bordermask[:,-3:-1] = 0
                bordermask = bordermask.astype(bool)
                crop_img = clear_border(np.array(crop_img, dtype=bool), mask = bordermask)

                # paste that cleaned up binary image into a new image with correct size
                result = np.full((new_image_height,new_image_width), color, dtype=np.float32)
                # compute center offset
                x_center = (new_image_width - old_image_width) // 2
                y_center = (new_image_height - old_image_height) // 2
                # copy img image into center of result image
                result[y_center:y_center+old_image_height, 
                   x_center:x_center+old_image_width] = crop_img

                result = np.array(result, dtype=bool)
                # append it into the seg_stack of the position object
                pos.rois[rois_nb].seg_stack.append(result.astype(np.uint8))

            # finally, we track!!
            pos.track(list(range(reader.timepoints)))
            # and extract features
            pos.features(frames=frames, features=features)
            # extract lineage object. If this is not empty, save .csv files
            lin = pos.rois[0].lineage
            if len(lin.cells)>0:
                df = delta_to_df(pos)
                save_name = save_directory + '/' + 'Pos' + str(round(meta.pos[i])).zfill(2) + chamber_folders[chami][-7:]
                #print(save_name)
                df.to_csv(save_name + '.csv' , index=False)
                if savemovie:
                    if saved_movies == savemoviefreq:
                        try:
                            pos.save(
                            filename=save_name,
                            frames=frames,
                            save_format=cfg.save_format,
                            )
                            saved_movies = 0
                        except Exception as e:
                            # Log any errors encountered during processing
                            fails.append(f"Error movie export {curr_dir},: {e}")
                    saved_movies +=1
        except Exception as e:
            # Log any errors encountered during processing
            fails.append(f"chamber {chami}: {e}")

    # re-read meta file (this ensures correct status as I run multiple scripts simultaneously that all read and write into it)
    meta = pd.read_csv(os.path.join(masterdir,metacsv))
    
    # update status and save
    meta.loc[i, ('delta')] = 'Done'
    if fails:
        meta.at[i, 'delta_fails'] = '; '.join(fails)
        
    meta.to_csv(os.path.join(masterdir,metacsv), index = False)
    today = date.today()
    meta.to_csv(os.path.join(masterdir,metacsv+'_' + today.strftime("%Y-%m-%d") + '.csv'), index = False)


r09, Pos 01:   0%|                                                                              | 0/27 [00:00<?, ?it/s]

r09, Pos 01:  15%|██████████▎                                                           | 4/27 [01:00<05:51, 15.28s/it]

# You are at the end!!

Now, have fun exploring your data with whatever other sofware you like :)